In [32]:
import streamlit as st
import pandas as pd
import numpy as np
import datetime
import requests
import json
import time
import urllib.parse
import urllib.request
from openai import OpenAI
import os
from dotenv import load_dotenv



In [33]:
def wiki_trending_today(n):
    """Grab n trending wikipedia articles from today
    Returns a tuple, (titles, extracts)"""
    today = datetime.datetime.now()
    date = today.strftime('%Y/%m/%d')
    language_code = 'en'

    load_dotenv()
    headers = {
    'Authorization': os.getenv('wiki_key'),
    'User-Agent': os.getenv('USER_AGENT')
    }
    base_url = 'https://api.wikimedia.org/feed/v1/wikipedia/'
    url = base_url + language_code + '/featured/' + date
    response = requests.get(url, headers=headers)
    response = json.loads(response.text)
    titles= []
    extracts=[]
    for i in response['mostread']['articles'][:n]:
        if 'description' in i.keys():
            titles.append(i['titles']['normalized'])
            extracts.append(i['extract'])
    return titles, extracts


In [61]:
def find_corresponding_news(titles, n, m):
    """given a list of titles, collect news articles corresponding to the first n titles
    Returns a list of strings, where each string can be used as a context for LLM
    """
    gnews_key= os.getenv("gnews_key")
    all_contexts=[]
    for title in titles[:n]:
        context=[]
        url = f'https://gnews.io/api/v4/search?q="{urllib.parse.quote(title)}"&lang=en&country=us&max={m}&apikey={gnews_key}'
        with urllib.request.urlopen(url) as res:
            data = json.loads(res.read().decode("utf-8"))
            for i in range(m):
                context.append(data["articles"][i]['content'])
        time.sleep(1)
        all_contexts.append(context)
    all_str_context=[]
    for context in all_contexts:
        strcontext=""
        for i in context:
            strcontext += i +'\n'
        all_str_context.append(strcontext)
    return all_str_context



In [53]:
def generate_questions(background, news):
    client = OpenAI()
    res=[]
    for b, n in zip(background, news):
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a beloved charismatic host of trivia at your local bar.Your job is to ask trivia questions using the context provided and then also provide the correct answer."},
                {"role": "system", "content": "Format your response like Q: question newline A: answer"},
                {"role": "system", "content": "Use this as context for your question:" + b + '\n' + n},
                {"role": "user", "content": "Based on the context, please ask your audience a question and provide the correct answer afterward."},
                {"role": "system", "content": "If your question contains themes of violence, simply respond 'pass' "},

            ]
        )
        print(completion.choices[0].message.content)
        res.append(completion.choices[0].message.content)
    return res



In [36]:
titles, extracts = wiki_trending_today(2)

In [62]:
print(titles)

['Pete Rose', 'Lyle and Erik Menendez']


In [63]:
news = find_corresponding_news(titles, 2, 2)
print(news)

['Longtime Pete Rose teammate Tony Perez said MLB’s hit king “wasn’t him at all” the day before he died Monday at the age of 83.\nPerez, who played alongside Rose with the Reds for 16 seasons, saw his friend Sunday during an autographs show in Tennessee... [2930 chars]\nLAS VEGAS (AP) — Betting on baseball got Pete Rose banned from the game, never to return while he was alive.\nBut, perhaps appropriately, he found a home in the nation’s gambling capital — where acceptance wasn’t a problem.\nIn choosing to live in Las ... [5432 chars]\n', '1 of 5 | Micaela Diamond and Nicholas Alexander Chavez appear in a scene from "Grotesquerie," airing on Thursday nights. Photo courtesy of FX/Hulu\nNEW YORK, Oct. 2 (UPI) -- Nicholas Alexander Chavez says it was a fascinating learning experience to wo... [4603 chars]\nTopline\nNetflix\'s dramatic series based on brothers Erik and Lyle Menendez and the murder of their parents drew more than 50% more viewers in its second week amid criticism from one br

In [64]:
print(news)

['Longtime Pete Rose teammate Tony Perez said MLB’s hit king “wasn’t him at all” the day before he died Monday at the age of 83.\nPerez, who played alongside Rose with the Reds for 16 seasons, saw his friend Sunday during an autographs show in Tennessee... [2930 chars]\nLAS VEGAS (AP) — Betting on baseball got Pete Rose banned from the game, never to return while he was alive.\nBut, perhaps appropriately, he found a home in the nation’s gambling capital — where acceptance wasn’t a problem.\nIn choosing to live in Las ... [5432 chars]\n', '1 of 5 | Micaela Diamond and Nicholas Alexander Chavez appear in a scene from "Grotesquerie," airing on Thursday nights. Photo courtesy of FX/Hulu\nNEW YORK, Oct. 2 (UPI) -- Nicholas Alexander Chavez says it was a fascinating learning experience to wo... [4603 chars]\nTopline\nNetflix\'s dramatic series based on brothers Erik and Lyle Menendez and the murder of their parents drew more than 50% more viewers in its second week amid criticism from one br

In [65]:
res =generate_questions(extracts, news)

Q: What nickname is Pete Rose famously known by, highlighting his hustle and tenacity on the baseball field?  
A: Charlie Hustle
Q: What are the names of the parents that the Menendez brothers were convicted of murdering?  
A: José and Mary Louise "Kitty" Menendez.


In [66]:
for QA in res:
    question, answer = QA.split('\nA: ')
    print(question)
    print(answer)

Q: What nickname is Pete Rose famously known by, highlighting his hustle and tenacity on the baseball field?  
Charlie Hustle
Q: What are the names of the parents that the Menendez brothers were convicted of murdering?  
José and Mary Louise "Kitty" Menendez.
